In [1]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

# Coleta dos Dados

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## Normalização das imagens

In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0

## Remodelagem (reshaping) a base de dados

In [5]:
# 6000 imagens, 28 de altura e 28 de largura
X_train.shape

(60000, 28, 28)

In [6]:
# altera de uma matriz para um vetor
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

# Criação de Clientes

In [7]:
def create_clients(X, y, n_clients):
    indexes = np.arange(X_train.shape[0])
    np.random.shuffle(indexes)
    X = X[indexes]
    y = y[indexes]
    
    rate = X.shape[0]//n_clients
    X_slices = np.array([X[(i *  rate) : ((i + 1) * rate)] for i in range(n_clients)])
    y_slices = np.array([y[(i *  rate) : ((i + 1) * rate)] for i in range(n_clients)])
    return X_slices, y_slices

In [8]:
X_slices, y_slices = create_clients(X_train, y_train, 10)

In [9]:
X_slices.shape, y_slices.shape

((10, 6000, 784), (10, 6000))

# Construção do Modelo

In [10]:
class MLP:
    def build(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
        return model

In [20]:
mlp = MLP()

global_model = mlp.build()
global_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

for j in range(8):
    print("Round ", j)
    
    global_weights = global_model.get_weights()
    weights = []
    
    for i in range(10):
        client_model = mlp.build()
        client_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
        
        client_model.set_weights(global_weights)
        X_slices, y_slices = create_clients(X_train, y_train, 10)
        
        client_model.fit(X_slices[i], y_slices[i], epochs=5)
        weights.append(client_model.get_weights())
    
    global_weights = weights[0]
    for i in range(1,10):
        global_weights += (X_slices[i].shape[0] / X_train.shape[0]) * np.ndarray(weights[i])
    
    global_model.set_weights(global_weights)

Round  0
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 1.2066 - sparse_categorical_accuracy: 0.5891
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 0.6021 - sparse_categorical_accuracy: 0.7856
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 0.5039 - sparse_categorical_accuracy: 0.8314
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4627 - sparse_categorical_accuracy: 0.8418
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4352 - sparse_categorical_accuracy: 0.8490
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 1.1564 - sparse_categorical_accuracy: 0.5935
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 0.5976 - sparse_categorical_accuracy: 0.7911
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 0.5101 - sparse_categorical_accuracy: 0.8213
Epoch 4/5
188/188 [============================

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


188/188 [==============================] - 0s 1ms/step - loss: 0.5865 - sparse_categorical_accuracy: 0.8409
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4191 - sparse_categorical_accuracy: 0.8423
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4009 - sparse_categorical_accuracy: 0.8466
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 0.3801 - sparse_categorical_accuracy: 0.8604
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 0.3600 - sparse_categorical_accuracy: 0.8621
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 0.6665 - sparse_categorical_accuracy: 0.8267
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4522 - sparse_categorical_accuracy: 0.8305
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4122 - sparse_categorical_accuracy: 0.8480
Epoch 4/5
188/188 [==============================] - 0s 1ms/step -

188/188 [==============================] - 0s 1ms/step - loss: 0.3617 - sparse_categorical_accuracy: 0.8650
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 0.5891 - sparse_categorical_accuracy: 0.8551
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4528 - sparse_categorical_accuracy: 0.8418
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4025 - sparse_categorical_accuracy: 0.8607
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 0.3821 - sparse_categorical_accuracy: 0.8638
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 0.3394 - sparse_categorical_accuracy: 0.8682
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 0.6284 - sparse_categorical_accuracy: 0.8283
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4517 - sparse_categorical_accuracy: 0.8517
Epoch 3/5
188/188 [==============================] - 0s 1ms/step -

188/188 [==============================] - 0s 1ms/step - loss: 0.5295 - sparse_categorical_accuracy: 0.8235
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4441 - sparse_categorical_accuracy: 0.8354
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 0.8640 - sparse_categorical_accuracy: 0.8156
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 0.6320 - sparse_categorical_accuracy: 0.8137
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 0.5051 - sparse_categorical_accuracy: 0.8244
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4658 - sparse_categorical_accuracy: 0.8255
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 0.4377 - sparse_categorical_accuracy: 0.8318
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 0.7766 - sparse_categorical_accuracy: 0.8287
Epoch 2/5
188/188 [==============================] - 0s 1ms/step -

188/188 [==============================] - 0s 1ms/step - loss: 1.5246 - sparse_categorical_accuracy: 0.7394
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 2.0757 - sparse_categorical_accuracy: 0.7457
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 1.2561 - sparse_categorical_accuracy: 0.7527
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 1.9419 - sparse_categorical_accuracy: 0.7726
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 1.5259 - sparse_categorical_accuracy: 0.7534
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 1.3725 - sparse_categorical_accuracy: 0.7489
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 1.2681 - sparse_categorical_accuracy: 0.7609
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 1.1135 - sparse_categorical_accuracy: 0.7514
Epoch 1/5
188/188 [==============================] - 0s 1ms/step -

188/188 [==============================] - 0s 1ms/step - loss: 3.5975 - sparse_categorical_accuracy: 0.7245
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 2.9731 - sparse_categorical_accuracy: 0.7059
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 2.6114 - sparse_categorical_accuracy: 0.7069
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 2.7668 - sparse_categorical_accuracy: 0.7136
Epoch 1/5
188/188 [==============================] - 0s 1ms/step - loss: 3.6567 - sparse_categorical_accuracy: 0.7599
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 3.2037 - sparse_categorical_accuracy: 0.7328
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 2.5465 - sparse_categorical_accuracy: 0.7370
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 2.6301 - sparse_categorical_accuracy: 0.7290
Epoch 5/5
188/188 [==============================] - 0s 1ms/step -

188/188 [==============================] - 0s 1ms/step - loss: 11.6338 - sparse_categorical_accuracy: 0.7128
Epoch 2/5
188/188 [==============================] - 0s 1ms/step - loss: 9.2589 - sparse_categorical_accuracy: 0.6963
Epoch 3/5
188/188 [==============================] - 0s 1ms/step - loss: 8.3099 - sparse_categorical_accuracy: 0.6720
Epoch 4/5
188/188 [==============================] - 0s 1ms/step - loss: 9.1823 - sparse_categorical_accuracy: 0.6907
Epoch 5/5
188/188 [==============================] - 0s 1ms/step - loss: 6.2949 - sparse_categorical_accuracy: 0.6920


In [21]:
test_loss, test_accuracy = global_model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 927us/step - loss: 41.9090 - sparse_categorical_accuracy: 0.7070


In [22]:
print("Loss: {:.3f}, Accuracy: {:.3f}".format(test_loss, test_accuracy))

Loss: 41.760, Accuracy: 0.706
